In [14]:
!pip install langchain_experimental
!pip install -U \
  langchain==0.3.14 \
  langchain-core==0.3.32 \
  langchain-community==0.3.14 \
  langchain-openai==0.2.14 \
  pydantic==2.9.2

  Using cached langchain_core-0.3.32-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_community-0.3.14-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached langsmith-0.2.11-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_text_splitters-0.3.10-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.7-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl.metadata (2.3 kB)
Using cached langchain_core-0.3.32-py3-none-any.whl (41

In [1]:
from langchain_experimental.utilities import PythonREPL

In [2]:
import os
from langchain_openai import AzureChatOpenAI

from dotenv import load_dotenv
load_dotenv("/content/.env")

## Start by creating an instance of the AzureChatOpenAI class.
chat = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    api_version="2024-12-01-preview",
    deployment_name="gpt-4.1",
    temperature=0,
)

In [3]:
# Tool: Python REPL for heavy math
from langchain.tools import Tool
python_tool = Tool(
    name="Python Calculator",
    func=PythonREPL().run,
    description="Useful for large computations (e.g., powers beyond 20). Input must be valid Python."
)

In [4]:
# Example query
query = "Calculate 10000 * 12**2 - 12 + log(10)"


In [5]:
tools = [python_tool]  # you can add wikipedia tool too


In [ ]:
# Other tools (optional, like Wikipedia)
#!pip install wikipedia
#import wikipedia
#from langchain_community.agent_toolkits.load_tools import load_tools
#other_tools = load_tools(["wikipedia"], llm=chat)
# Combine tools
#tools = [python_tool] + other_tools

In [6]:
#Create ReAct prompt
from langchain import hub

# Standard ReAct prompt for reasoning + tool usage
prompt = hub.pull("hwchase17/react")


/usr/local/lib/python3.12/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
import langchain
import langchain_core
import langchain_community
import pydantic

print("langchain:", langchain.__version__)
print("langchain_core:", langchain_core.__version__)
print("langchain_community:", langchain_community.__version__)
print("pydantic:", pydantic.__version__)

langchain: 0.3.14
langchain_core: 0.3.32
langchain_community: 0.3.14
pydantic: 2.9.2


In [8]:
from langchain.agents import create_react_agent, AgentExecutor
agent = create_react_agent(
    llm=chat,
    tools=tools,
    prompt=prompt
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [9]:
# Query example
result = executor.invoke({"input": "Calculate 10000 * 12**2 - 12 + log(10)"})
print(result["output"])



> Entering new AgentExecutor chain...


Thought: First, I need to calculate 12 squared, then multiply by 10000, subtract 12, and finally add the logarithm of 10. I'll use the Python Calculator for this computation.
Action: Python Calculator
Action Input: 10000 * 12**2 - 12 + log(10)NameError("name 'log' is not defined")Thought: The error occurred because the log function is not defined. I need to use the math module's log function, and since the base is not specified, I'll assume it's the natural logarithm. I'll import the math module and use math.log(10).
Action: Python Calculator
Action Input: import math; 10000 * 12**2 - 12 + math.log(10)Thought: I did not receive the result from the last calculation. I will try again, ensuring the correct use of the math module for the logarithm.
Action: Python Calculator
Action Input: import math; 10000 * 12**2 - 12 + math.log(10)Thought: It seems the Python Calculator may not return output when multiple statements are used. I should break the calculation into a single expression, using

In [11]:
# Query example
result = executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...
First, I need to calculate 12 squared, then multiply by 10,000, subtract 12, and add the logarithm of 10 (assuming natural log).
Action: 10000 * 12**2 - 12 + __import__('math').log(10)Invalid Format: Missing 'Action Input:' after 'Action:'Action: Python Calculator
Action Input: 10000 * 12**2 - 12 + __import__('math').log(10)Thought: The previous step did not return a result. I will try again, ensuring the calculation is correct and using the Python Calculator tool.
Action: Python Calculator
Action Input: 10000 * 12**2 - 12 + __import__('math').log(10)Thought: The previous attempts did not return a result. I will break down the calculation step by step to ensure accuracy.

Step 1: Calculate 12 squared.
Step 2: Multiply the result by 10,000.
Step 3: Subtract 12.
Step 4: Add the natural logarithm of 10.

Let me compute each step using the Python Calculator.

Action: Python Calculator
Action Input: 12**2Action: Python Calculator
Action Input: 12**2Th

In [14]:
#Using newer approach
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnableSequence

chat = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    api_version="2024-12-01-preview",
    deployment_name="gpt-4.1",
    temperature=0,
)

# Example tools as Python functions
def echo_tool(input_text: str) -> str:
    return f"Echo: {input_text}"

def reverse_tool(input_text: str) -> str:
    return input_text[::-1]

# Agent sequence
def agent_step(user_input):
    # Decide which tool to use
    if "reverse" in user_input.lower():
        return reverse_tool(user_input)
    else:
        return echo_tool(user_input)

# Memory
chat_history = []

def add_memory(role, msg):
    chat_history.append({"role": role, "content": msg})

# Multi-step runnable sequence
def remember_step(user_input):
    add_memory("user", user_input)
    return user_input

def llm_step(user_input):
    # Minimal safe prompt
    prompt = f"You are a helpful assistant. User says: {user_input}. Respond helpfully."
    response = chat.invoke(prompt)
    add_memory("assistant", response.content)
    tool_result = agent_step(user_input)
    return f"{response.content}\nTool result: {tool_result}"

from langchain_core.runnables import RunnableSequence
sequence = RunnableSequence(remember_step, llm_step)

output = sequence.invoke("Hello agent, reverse this text!")
print(output)


Of course! Please provide the text you'd like me to reverse.
Tool result: !txet siht esrever ,tnega olleH
